Summary:
- Looking at D2S + D2D in FL model training
- D2D clusters = time varying + directed communication graphs
- Algorithm -> controls trade-off between 
    - rate of convergence to global optimizer
    - number of D2S transmissions required for global aggregation
- Main point: D2D updates injected into FL averaging framework based on column-stochastic weight matrices that encapsulate the connectivity within clusters
    - Basically the connectivity matrices in clusters decide on when to send updates
- Expected optimality gap (between current global model and optimal global model) depends on 2 biggest singular values of weighted adjacency matrices of the D2D clusters

Key contributions:
- Each D2D cluster is time-varying digraph -> the expected optimality gap depends on 2 greatest singular values of the weighted adjacency matrices for local aggregations in the clusters
- Singular value bounds in terms of node degrees -> the singular values are bounded by the degree distribution of every cluster
- Connectivity-Aware Learning Algorithm -> Singular value bounds are used to design a time-varying threshold on the number of clients required to be sampled by server for global aggregation to enforce a specified convergence rate while reducing number of D2S communications
- Effect of data heterogeneity under mild gradient diversity assumptions -> the expected optimality gap is bounded by a value that captures cluster densities and data heterogeneity across the devices

Notation:
- $\mathbf{v} \in \mathbb{R} :=$ stochastic $ \iff \mathbf{v} \geq 0 (\forall i, v_i \geq 0)$ and $ \mathbf{v}^T\mathbf{1} = 1$ (i.e. $[v_1, v_2, \ldots, v_n][1, 1, \ldots, 1]^T = 1$) (so the sum of vector components sums up to 1 and all are greater than 0)
- A is a column stochastic matrix $\iff A > 0$ (for all elements) and each column of A sums up to 1: $A^T\mathbf{1} = \mathbf{1} \Rightarrow $ $ A \in \mathbb{R}^{m \times n}, \left[\begin{array}{c}
\mathbf{col}_1^T\\
\mathbf{col}_2^T \\
\vdots \\
\mathbf{col}_n^T
\end{array}\right] \left[\begin{array}{c}
1\\
1 \\
\vdots \\
1
\end{array}\right] = \left[\begin{array}{c}
1\\
1 \\
\vdots \\
1
\end{array}\right]$
- A is symmetric if $A = A^T$
- Consensus matrix = influence of nodes in a network, ex: $\left[\begin{array}{ccc}
c_{1, 1} & \ldots & \vdots \\
c_{2, 1} & \ddots & \vdots \\
\vdots & \ldots & \vdots
\end{array}\right] \Rightarrow c_{i,j}$ is how node i influences node j 

Assumptions:
- D2D network communications are NOT necessarily bidirectional (cluster graphs are directed)
- Column-stochastic consensus matrices need not be symmetric

Technical Challenges:
1. Cannot use standard eigenvalue results in the analysis (must use singular values)
2. Column stochastic aggregation matrices do NOT ensure convergence to consensus in the absence of central coordinatior -> the analysis must account for combined effect of global aggregation + column stochasticity


Semi-Decentralized FL Setup:
- $n$ local clients + 1 central parameter server (PS) that aggregates all local models
- $[n] :=$ set of clients
- $D_i :=$ local dataset of each client $i \in [n]$
- $\xi = (u,y) \in D_i$ is a data sample, where $u \in \mathbb{R}^p$ is a feature vector of the sample, and $y$ is its label
- $x$ is a model
- Loss function $L$ is defined as $L: \mathbb{R}^p \times {\cup}_{i = 1}^nD_i \rightarrow \mathbb{R}$, so that $L(x, \xi)$ denotes loss incurred by $x$ on a sample $\xi \in {\cup}_{i=1}^nD_i$
    - Note: ${\cup}_{i=1}^nD_i$ is the global dataset
    - Also $x \in {\R}^p$ because we flatten the model into a vector
- Average loss incurred by model $x$ over local dataset of client $i$ is: $f_i(x) := \frac{1}{|D_i|}{\sum}_{\xi \in D_i}L(x, \xi)$
    - $f_i :=$ local loss function of client i
- Clients seek to minimize global loss function $f := \frac{1}{n}{\sum}_{i = 1}^nf_i(x)$
- Learning objective: find global optimum: $x^* := \underset{x}{argmin}f(x)$

D2D and D2S Network Models:
- D2S interactions -> devices can send to PS if prompted by it
- D2D network = time-varying directed graph $G(t) = ([n], E(t))$, where $[n] := $ vertex set and $E(t) := $ edge set of the digraph
    - if a directed edge from node $i \in [n]$ to another node $j \in [n]$ exists in $G(t)$, then there is a communication link from $i$ to $j$ in the D2D network
    - $i$ = in-neighbor of $j$
    - $j$ = out-neighbor of $i$
    - $\mathcal{N}_i^-(t) :=$ set of in-neighbors of client $i \in [n]$ at time $t$
    - $\mathcal{N}_i^+(t) :=$ set of out-neighbors of client $i \in [n]$ at time $t$
    - $d_i^-(t)$ = in-degree = number of in-neighbors of node $i$
    - $d_i^+(t)$ = out-degree = number of out-neighbors of node $i$
    - $d_{max}^-(t)$ = max in-degree
    - $d_{min}^+(t)$ = min out-degree
    - $d_{max}^+(t)$ = max out-degree
- Assumptions for D2D:
    - Not strongly/uniformly connected over time
    - So there is a number $c > 1$ of strongly connected components (strongly connected components = set of vertices where we can get between any 2 from one to the other) of $G(t)$ denoted $\{(V_1(t), E_1(t)), (V_2(t), E_2(t)), \ldots, (V_c(t), E_c(t))\}$ = clusters of the D2D network
    - $c$ = number of clusters is time invariant
    - There is no communication link between any 2 clusters. So, $E(t) = {\cup}_{l=1}^cE_l(t)$
    - The server has full knowledge of the vertex sets $\{V_l(t)\}_{l=1}^c$ at any time $t$

Proposed Method:
- Local model updates:
    - $x^{(t)} :=$ global model at the end of t-th round
    - Each client $i \in [n]$ performs $T \in \N$ iterations of local SGD
    - So for each $k \in \{0, 1, \ldots, T-1\}$: $x_i^{(t, k+1)} = x_i^{(t, k)}-{\eta}_t\overset{\sim}{\nabla} f_i(x_i^{(t,k)})$
    - ${\eta}_t > 0$ is the learning rate (= step size)
    - $\overset{\sim}{\nabla} f_i(x) := \frac{1}{|X_i|}{\nabla}{\sum}_{\xi \in X_i}L(x;\xi)$ = stochastic gradient computed by client $i$ on a mini-batch $X_i \subset D_i$ of the local samples
    - Note: $x_i^{t,0} := x^{(t)}$
- Intra-Cluster model aggregations (within one cluster):
    - Every client $i \in [n]$ transmits its scaled cumulative stochastic gradient $x_i^{(t, T)}-x^{(t)}=-{\eta}_t{\sum}_{k=0}^{T-1}\overset{\sim}{\nabla}f_i(x_i^{(t,k)})$ to each of its out-neighbors $j \in \mathcal{N}_i^+(t)$
    - This is done before t-th global aggregation round
    - Assume: Every cluster $l \in [c]$ contains an access point to which every client $i \in V_l(t)$ sends a list of its in-neighbors (clients whose gradients i has received)
    - The access point at the end announces the end of the concerned D2D communication round
    - The access point determines: $\{d_j^+(t):j \in V_l(t)\} :=$ out-degree for each client in the cluster
    - The access point broadcasts this sequence to every client in the cluster
    - Then each client computes a weighted sum of all scaled cumulative gradients from its in-neighbors:
        - ${\Delta}_i(t)=\underset{j \in \mathcal{N}_i^-(t)}{\sum}\frac{1}{d_j^+(t)}(x_j^{(t,T)}-x^{(t)})$ 
        - So the in-neighbor stochastic gradients are scaled by their out-degrees 
        - <font color='red'>Question </font>: Why choose to scale by out-degrees? Is this standard? Or is there some reason behind it? Why not scale by the size of the dataset of each client?
            - Want to reach global optimum = unweighted average of global loss funtions
    - So this rule can be expressed in a matrix form as:
        - $\mathbf{{\Delta}}(t) = A(t)X_{diff}^T(t)$ where:
            - $\mathbf{{\Delta}}(t)  := [{\Delta}_1(t), {\Delta}_2(t), \ldots, {\Delta}_n(t)]^T$
            - $X_{diff}(t) = [x_1^{(t,T)}-x^{(t)},  x_2^{(t,T)}-x^{(t)}, \ldots,  x_n^{(t,T)}-x^{(t)}]$
            - $A(t) \in {\R}^{n \times n}$ = matrix whose $(i,j)$-th  entry is: $a_{i,j}(t) = \frac{1}{d_j^+(t)}$ <font color='red'>if j is in in-neighbors of i and 0 else</font> (is that correct? Or did I miss something?) for all $i,j \in [n]$
        - Fact 1. $A(t)$ is column stochastic:
            - <font color='red'>Pf: </font> $\underset{i=1}{\overset{n}{\sum}} a_{i,j}(t) = \underset{i \in [n]: j \in \mathcal{N_i^-(t)}}{\sum} \frac{1}{d_j^+(t)} = \underset{i \in \mathcal{N_j^+(t)}}{\sum} \frac{1}{d_j^+(t)} = \underset{i \in \mathcal{N_j^+(t)}}{\sum} \frac{1}{|\mathcal{N_j^+(t)}|} = 1$
            - Basically, each cloumn just stores the out-neighbors for a given client, and each out-neighbor is weighted as $\frac{1}{d_j^+(t)}$, so the sum over each column is 1
        - We refer to $A(t)$ as equal-neighbor adjacency matrix of $G(t)$, since every client $i \in [n]$ transmits an equal share ($\frac{1}{d_i^+(t)}$) of its scaled cumulative gradient to its ${d_i^+(t)}$ out-neighbors
- Global Aggregation at the PS
    - PS samples a random subset of clients $S(t) \subset [n]$
    - $|S(t)| = m(t) \leq n$ is chosen by the algorithm to complement intra-cluster aggregations without excessively slowing down the training process
    - Specifically 3 steps:
        1. PS learns degree distribution of each cluster (distribution of out-degrees for each node in cluster)
        2. PS computes upper bound on an error quantity $\phi(t) :=$ combined effect of random sampling and the cluster degree distributions on the convergence rate
        3. PS computes minimum value of $m(t)$ required to keep $\phi(t)$ below a desired threshold
    - For the $(t+1)$-th round of global aggregation:
        1. Server uses $m(t)$ (computed in previous iteration) to select $\lceil (\frac{m(t)}{n})n_l(t)\rceil$ clients uniformly at random from the $n_l(t) := |V_l(t)|$ clients that make cluster $l \in [c]$ (so every cluster has a representation proportionate to its size in the global aggregation)
        2. Server updates global model: $x^{(t+1)} = x^{(t)}+\frac{1}{m(t)}\underset{i \in S(t)} {\sum}{\Delta}_i(t)=\frac{1}{m(t)}\underset{i = 1} {\overset{n}{\sum}}{\tau}_i(t){\Delta}_i(t)$, where ${\tau}_i(t) := |\{i\} \cap S(t)|$ is an indicator random variable that takes value 1 if $i$ is sampled and 0 else. Note: ${\sum}_{i=1}^n{\tau}_i(t)=|S(t)|=m(t)$
        3. The current round now is $t \leftarrow t + 1$. All cluster access points send cluster out-degree sequences to the server
        4. Server computes ${\alpha}_l(t) := \frac{1}{|V_l(t)|}{min}_{i \in V_l(t)}d_i^+(t)$ = minimum out-degree fraction of cluster $l \in [c]$
        5. The server uses either of the two sets of singular value bounds (derived later) to compute an upper bound $\Psi(m(t), {\alpha}_1(t), \ldots, {\alpha}_c(t))$ on the connectivity factor affecting the convergence rate.
            - Note: The connectivity factor is defined as: ${\phi}(t) := (\frac{n}{m(t)} - 1)\underset{l=1}{\overset{c}{\sum}}\frac{|V_l(t)|}{n}{\phi}_l(t)$ where ${\phi}_l(t) := {\sigma}_1^2(A_l(t))+{\sigma}_2^2(A_l(t)) - 1$ depends on the 2 singular values ${\sigma}_1(A_l(t)) > {\sigma}_2(A_l(t))$ of matrix $A_l(t)$ of cluster $l$
            - Note: It will be shown that: $\Psi(m(t), {\alpha}_1(t), \ldots, {\alpha}_c(t)) := (\frac{n}{m(t)} - 1)\underset{l=1}{\overset{c}{\sum}}\frac{|V_l(t)|}{n}{\Psi}_l(t)$
            - Note: either of the following holds ((t) indexing omitted on the right side):
                - ${\Psi}_l(t) = 1 + {\varepsilon}_l + (\frac{1}{{\alpha}_l}-1)^2+2{\varepsilon}_l(1 + \frac{2}{{\alpha}_l} - \frac{1}{{\alpha}_l^2})$
                - ${\Psi}_l(t) = 2 + 2{\varphi}_l-\frac{(1-{\varepsilon}_l)^2(1-{\alpha}_{-l}^2)((1-{\varepsilon}_l)^2(1-{\alpha}_{-l}^2)-{\alpha}_{-l})}{|V_l(t)|({\varepsilon}_{net,l}+1)({\varepsilon}_{net,l}-{\alpha}_{-l}+\frac{1}{{\alpha}_{l}|V_l(t)|})}$ with
                    - ${\varepsilon}_l(t):= \frac{d_{max}^+(t) - d_{min}^+(t)}{d_{min}^+(t)}$
                    - ${\varphi}_l(t):= \frac{d_{max}^-(t) - d_{min}^+(t)}{d_{min}^+(t)}$
                    - ${\alpha}_{-l}(t):=\frac{1}{{\alpha}_{l}(t)}-1$
                    - ${\varepsilon}_{net,l}(t) = {\varphi}_l(t)+\frac{{\varepsilon}_l(t)}{{\alpha}_{l}(t)}$
        6. Finally, the server sets: $m(t+1):=min\{r \in [n]:\Psi(r, {\alpha}_1(t), \ldots, {\alpha}_c(t)) \leq {\phi}_{max}\}$ where ${\phi}_{max}$ is a threshold given as an input to the algorithm.


Convergence Analysis
- Assumptions and Preliminaries
    1. Assumption 1 (Strong Convexity):
        - First of all, the strong convexity assumption assumes that a given function is not only convex (i.e. curve upwards everywhere), but also the rate of the at which it curves increases away from the global minimum. This guarantees faster convergence to global minimum
        - The condition is as follows: All the local loss functions $\{f_i\}_{i=1}^n$ are $\mu$-strongly convex, i.e. there exists $\mu > 0$ such that $(\nabla f_i(x)-\nabla f_i(y))^T(x-y) \geq \mu ||x-y||^2$ for all $x,y \in {\R}^p$ and all $i \in [n]$
        - Read this as follows: gradient (slope) difference between 2 points in the direction of these points should be higher than their quadratic difference
        - I.e. as I move from $y$ to $x$, the bigger the distance squared, the bigger should the slope difference be
        - So bigger difference in distance between 2 points means the slope difference is bigger
    2. Assumption 2 (Smoothness): 
        - Essentially, smoothness bounds the max gradient change (so slope change in every plane) so that there are no "jumps" anywhere
        - Mathematically: All the local loss functions $\{f_i\}_{i=1}^n$ are $\beta$-smooth, i.e. there exists a finite $\beta$ such that $||(\nabla f_i(x)-\nabla f_i(y))|| \leq \beta ||x-y||$ for all $x,y \in {\R}^p$ and all $i \in [n]$
    3. Assumption 3 (Unbiasedness and bounded variance):
        - The SGD noise associated with every client is unbiased, i.e.: $\mathbb{E}[\overset{\sim}{\nabla} f_i(x) - {\nabla} f_i(x)|x] = 0$, and has bounded variance, i.e. there exists a constant $\varrho > 0$ such that $\mathbb{E}||\overset{\sim}{\nabla} f_i(x) - {\nabla} f_i(x)||^2 \leq {\varrho}^2$ for all models $x$
        - The first part is saying that given any model $x$, the expected value of stochastic gradient will be the same as true gradient
        - The second part is saying that for any model $x$, the expected difference between stochastic gradient and true gradient is less than some positive constant
        - Also, on top of the two above, there is an assumption that the SGD noise is independent across clients
    4. Assumption 4 (Gradient Diversity):
        - This assumes training data are not distributed uniformly among the clients
        - So data is heterogeneous among the clients
        - Mathematically: For all $i \in [n]$ and $x$, we have $||{\nabla} f_i(x) - {\nabla} f(x)|| \leq \delta + 2 \beta ||x - x^*||$, where $ \delta := \beta \underset{i \in [n]}{max}||x^*-x_i^*|| = \beta \underset{i \in [n]}{max}||x^*-\underset{y}{argmin}f_i(y)||$
        - The left side of the inequality is the magnitude of the difference between client gradient and global gradient for current global model
        - The right side has 2 components:
            - $\delta := \beta \underset{i \in [n]}{max}||x^*-x_i^*||$ - this is the max difference between global optimum, and best client optimum models
            - $ 2 \beta ||x - x^*||$ - this is just the difference between current global model and global optimum
        - So this condition bounds the difference of gradients for each client and global gradient with max client-global optimal model difference, and the current global model-optimal model difference

Results and Proofs:
    
Lemma 4.2: At the end of $(t+1)$-th round of global aggregation, the expected optimality gap of the proposed algorithm is: 
\begin{equation}
    \mathbb{E}||x^{(t+1)}-x^*||^2=\mathbb{E}||x^{(t+1)}-\bar{x}^{(t+1)}||^2+\mathbb{E}||\bar{x}^{(t+1)}-x^2||^2
\end{equation}
where $ \bar{x}^{(t+1)} := x^{(t)}+\frac{1}{n}{\sum}_{i=1}^n(x_i^{(t,T)}-x^{(t)})$ is a vector that would equal global model if all clients were sampled (standard FL)

<font color='red'>Pf</font>
1. $ \mathbb{E}||x^{(t+1)}-x^*||^2 = \mathbb{E}||x^{(t+1)} - \bar{x}^{(t+1)} + \bar{x}^{(t+1)} -x^*||^2 = \mathbb{E}||(x^{(t+1)} - \bar{x}^{(t+1)}) + (\bar{x}^{(t+1)} -x^*)||^2$
2. $\mathbb{E}||(x^{(t+1)} - \bar{x}^{(t+1)}) + (\bar{x}^{(t+1)} -x^*)||^2$ =  $\mathbb{E}[((x^{(t+1)} - \bar{x}^{(t+1)}) + (\bar{x}^{(t+1)} -x^*))^T((x^{(t+1)} - \bar{x}^{(t+1)}) + (\bar{x}^{(t+1)} -x^*))]$

3. Mutliplying the terms inside I get:
\begin{align*}
    \mathbb{E}||x^{(t+1)}-x^*||^2 = \mathbb{E}||x^{(t+1)} - \bar{x}^{(t+1)}||^2 + \mathbb{E}|| \bar{x}^{(t+1)} -x^*||^2 +\\
     \mathbb{E}[(x^{(t+1)} - \bar{x}^{(t+1)})^T(\bar{x}^{(t+1)} -x^*)] + \mathbb{E}[(\bar{x}^{(t+1)} -x^*)^T(x^{(t+1)} - \bar{x}^{(t+1)})]
\end{align*}
4. So now I only need to show that $\mathbb{E}[(x^{(t+1)} - \bar{x}^{(t+1)})^T(\bar{x}^{(t+1)} -x^*)] + \mathbb{E}[(\bar{x}^{(t+1)} -x^*)^T(x^{(t+1)} - \bar{x}^{(t+1)})] = 0$
5. First, I can write: $(x^{(t+1)} - \bar{x}^{(t+1)})^T(\bar{x}^{(t+1)} -x^*) =(x^{(t+1)} - \bar{x}^{(t+1)}) \cdot (\bar{x}^{(t+1)} -x^*)$
6. For the second expectation the value inside is: $(\bar{x}^{(t+1)} -x^*)^T(x^{(t+1)} - \bar{x}^{(t+1)}) = (\bar{x}^{(t+1)} -x^*) \cdot (x^{(t+1)} - \bar{x}^{(t+1)})$
7. Since dot product is commutative, the two values are equal, so this shows that 
\begin{align*}
    \mathbb{E}[(x^{(t+1)} - \bar{x}^{(t+1)})^T(\bar{x}^{(t+1)} -x^*)] = \mathbb{E}[(\bar{x}^{(t+1)} -x^*)^T(x^{(t+1)} - \bar{x}^{(t+1)})]
\end{align*}
8. Next, notice that we can write $x^{(t+1)} - \bar{x}^{(t+1)} = (x^{(t+1)} - x^{(t)}) - (\bar{x}^{(t+1)} - x^{(t)}) = (x^{(t)} + \frac{1}{m} {\sum}_{i \in S(t)} {\Delta}_i(t)- x^{(t)}) - (x^{(t)}+\frac{1}{n}{\sum}_{i=1}^n(x_i^{(t,T)}-x^{(t)}) - x^{(t)}) = \frac{1}{m}{\sum}_{i \in S(t)} {\sum}_{j \in \mathcal{N}_i^-(t)}\frac{1}{d_j^+(t)}(x_j^{(t,T)}-x^{(t)})- \frac{1}{n}{\sum}_{i=1}^n(x_i^{(t,T)}-x^{(t)})$
9. I can write $\frac{1}{n}{\sum}_{i=1}^n(x_i^{(t,T)}-x^{(t)})$ as $\frac{1}{n}X_{diff}\mathbf{1}$
10. I can write $\frac{1}{m}{\sum}_{i \in S(t)} {\sum}_{j \in \mathcal{N}_i^-(t)}\frac{1}{d_j^+(t)}(x_j^{(t,T)}-x^{(t)})$ as $v(t)A(t)X_{diff}^T(t)$
    - <font color='red'>Questions:</font>:
        - Firstly, where does the form $X_{diff}v^T(t)A(t)$ come from?
        - For the second one, why not $\frac{1}{n}X_{diff}\mathbf{1}$
        - Also, why is the transpose stil there in line 2? I thought it was applied
        - Why is $\mathbb{E}[v(t)] = \frac{1}{n}\mathbf{1}$? -> Actually make sure this is correct: $E[v_1(t)] = {\sum}_{i = 1}^n v_1 p_V(v_1) = {\sum}_{i \in S(t)} \frac{1}{m} \frac{1}{n} = \frac{1}{mn} {\sum}_{i \in S(t)}1 = \frac{1}{mn} m = \frac{1}{n}$. That holds for all dimensions in $v(t)$

Proposition 4.3 and auxilary lemmas

Lemma 3: For any set of $k \in \N$ random vectors $\{Y^{(i)}\}_{i=1}^k$ that ake values in ${\R}^q$ (where $q \in \N$) we have: $\mathbb{E}||{\sum}_{i=1}^kY^{(i)}||^2 \leq k{\sum}_{i=1}^k \mathbb{E}||Y^{(i)}||^2$

<font color='red'>Pf</font> (this one was easy btw so skip):
1. Let $\hat{Y}^{(j)} := [Y_1^{(1)}, Y_j^{(2)}, \ldots, Y_j^{(k)}]^T$ for each $j \in [q]$
2. $\mathbb{E}||{\sum}_{i=1}^kY^{(i)}||^2 = \mathbb{E}||\begin{bmatrix}
{\sum}_{i=1}^kY_1^{(i)} \\
{\sum}_{i=1}^kY_2^{(i)} \\
\vdots \\
{\sum}_{i=1}^kY_m^{(i)} \\
\end{bmatrix}
||^2 = \mathbb{E}[\begin{bmatrix}
{\sum}_{i=1}^kY_1^{(i)} & {\sum}_{i=1}^kY_2^{(i)} & \ldots & {\sum}_{i=1}^kY_m^{(i)}
\end{bmatrix}
\begin{bmatrix}
{\sum}_{i=1}^kY_1^{(i)} \\
{\sum}_{i=1}^kY_2^{(i)} \\
\vdots \\
{\sum}_{i=1}^kY_m^{(i)} \\
\end{bmatrix}] = E[{\sum}_{j=1}^m({\sum}_{i=1}^kY_j^{(i)})^2]$
3. $E[{\sum}_{j=1}^m({\sum}_{i=1}^kY_j^{(i)})^2] = {\sum}_{j=1}^mE[({\sum}_{i=1}^kY_j^{(i)})^2] = {\sum}_{j=1}^mE[(\mathbf{1}^T\hat{Y}^{(j)})^2]$
4. ${\sum}_{j=1}^mE[(\mathbf{1}^T\hat{Y}^{(j)})^2] = {\sum}_{j=1}^mE[(\mathbf{1} \cdot \hat{Y}^{(j)})^2]$
5. By the Cauchy schwarz inequality ($(\mathbf{v} \cdot \mathbf{u})^2 \leq (\mathbf{v} \cdot \mathbf{v}) (\mathbf{u} \cdot \mathbf{u})$): ${\sum}_{j=1}^mE[(\mathbf{1} \cdot \hat{Y}^{(j)})^2] \leq {\sum}_{j=1}^mE[(||\mathbf{1}||^2||\hat{Y}^{(j)})||^2]$
6. Also note that $||\mathbf{1}||^2 = k$
7. So ${\sum}_{j=1}^mE[(\mathbf{1}^T\hat{Y}^{(j)})^2] \leq {\sum}_{j=1}^mE[(||\mathbf{1}||^2||\hat{Y}^{(j)})||^2] = k{\sum}_{j=1}^mE||\hat{Y}^{(j)}||^2 = k{\sum}_{j=1}^mE(\hat{Y}^{(j)T} \hat{Y}^{(j)}) = k{\sum}_{j=1}^mE({\sum}_{i=1}^kY_j^{(i)})$
8. Finally: ${\sum}_{j=1}^mE[(\mathbf{1}^T\hat{Y}^{(j)})^2] \leq k{\sum}_{j=1}^mE({\sum}_{i=1}^kY_j^{(i)}) = kE({\sum}_{i=1}^k{\sum}_{j=1}^m(Y_j^{(i)})^2) = k{\sum}_{i=1}^kE({\sum}_{j=1}^m(Y_j^{(i)})^2) = k{\sum}_{i=1}^kE||Y^{(i)}||^2$ QED

Lemma 4: Every $n \times n$ matrix $\mathbf{W}$ satisfies: $||\mathbf{W}||^2 \leq {\sum}_{i=1}^n||W_{:,i}||^2$, where $\{W_{:,i}\}_{i=1}^n$ are the columns of $\mathbf{W}$

<font color='red'>Pf</font>
1. Note that $||\mathbf{W}||^2$ is a spectral norm of the matrix $\mathbf{W}$, i.e.: -> follows from definition, but ask
- Isn't the trace supposed to be applied to the right? Maybe just ask about the flow:
${\sum}_{i=1}^n||W_{:,i}||^2 = trace(\mathbf{W}^T\mathbf{W})={\sum}_{i=1}^n {\sigma}_i^2(\mathbf{W}) \geq {\sigma}_1^2(\mathbf{W}) = ?$ (this point?)

Lemma 5: For $s \in \N$ let $A \in {\R}^{s \times s}$ be an irreducible non-negative matrix with positive diagonal entries. Then matrix $AA^T$ is irreducible

<font color='red'>Pf</font>

Note that a matrix is irreducible if it creates a strongly connected digraph (i.e. can get from any vertex to any other vertex)
1. For any 2 indices $i, j \in [s]$, we have $(AA^T)_{i,j} = {\sum}_{k=1}^s a_{i,k}a_{j,k} > a_{i,j}a_{j,j}$ <font color='red'>(why is that true?)</font>
2. Since we assumed $a_{j,j}$ are all positive, then $(AA^T)_{i,j} > 0$ whenever $ a_{i,j} > 0$
3. So $G(AA^T)$ is a super-digraph of $G(A)$, and $G(A)$ is a strongly connected digraph because $A$ is irreducible
4. So $G(AA^T)$ is strongly connected, so $AA^T$ is irreducible, QED

Lemma 6: For $s \in \N$ let $v \in {\R}^s$ be a stochastic vector, and let $A \in {\R}^{s \times s}$ be an irreducible column-stochastic matrix with positive diagonal entries. Then: $||A^Tv-\frac{1}{s}\mathbf{1}||^2 \leq ({\sigma}_1^2+{\sigma}_2^2-1)||v_{\perp}||^2$, where $v_{\perp} := v - \frac{1}{s}\mathbf{1}$ is the component of $v$ orthogonal $\mathbf{1}$ and ${\sigma}_1$ and ${\sigma}_2$ are largest and second-largest singular values of $A$

<font color='red'>Pf</font>

1. $||A^Tv-\frac{1}{s}\mathbf{1}||^2 = (A^Tv-\frac{1}{s}\mathbf{1})^T(A^Tv-\frac{1}{s}\mathbf{1}) = (v^TA-\frac{1}{s}\mathbf{1}^T)(A^Tv-\frac{1}{s}\mathbf{1}) = (v^TAA^Tv - \frac{1}{s}\mathbf{1}^TA^Tv - v^TA\frac{1}{s}\mathbf{1} + \frac{1}{s^2}\mathbf{1}^T\mathbf{1})$
2. $(v^TAA^Tv - \frac{1}{s}\mathbf{1}^TA^Tv - v^TA\frac{1}{s}\mathbf{1} + \frac{1}{s^2}\mathbf{1}^T\mathbf{1}) = (v^TAA^Tv - \frac{2}{s}\mathbf{1}^TA^Tv - + \frac{1}{s^2}\mathbf{1}^T\mathbf{1})$
3. 

\begin{gather*}
    (v^TAA^Tv - \frac{2}{s}\mathbf{1}^TA^Tv + \frac{1}{s^2}\mathbf{1}^T\mathbf{1}) = (v_{\perp}+\frac{1}{s}\mathbf{1})^TAA^T(v_{\perp}+\frac{1}{s}\mathbf{1}) - \frac{2}{s}\mathbf{1}^TA^T(v_{\perp}+\frac{1}{s}\mathbf{1}) + \frac{1}{s^2}\mathbf{1}^T\mathbf{1} \\
    = v_{\perp}^TAA^Tv_{\perp} + \frac{1}{s^2} (A^T\mathbf{1})^TA^T\mathbf{1} + \frac{2}{s}(A^T\mathbf{1})^TA^Tv_{\perp} - \frac{2}{s}\mathbf{1}^TA^Tv_{\perp} - \frac{2}{s^2}\mathbf{1}^TA^T\mathbf{1}+\frac{1}{s^2}\mathbf{1}^T\mathbf{1}
\end{gather*}
4. Since $A$ is column stochastic, $A^T\mathbf{1} = \mathbf{1}$, so I can simplify the above equation:
\begin{gather*}
    v_{\perp}^TAA^Tv_{\perp} + \frac{1}{s^2} (A^T\mathbf{1})^TA^T\mathbf{1} + \frac{2}{s}(A^T\mathbf{1})^TA^Tv_{\perp} - \frac{2}{s}\mathbf{1}^TA^Tv_{\perp} - \frac{2}{s^2}\mathbf{1}^TA^T\mathbf{1}+\frac{1}{s^2}\mathbf{1}^T\mathbf{1}\\
    = v_{\perp}^TAA^Tv_{\perp} + \frac{1}{s} + \frac{2}{s}\mathbf{1}^TA^Tv_{\perp} - \frac{2}{s}\mathbf{1}^TA^Tv_{\perp} - \frac{2}{s} + \frac{1}{s} \\
    = v_{\perp}^TAA^Tv_{\perp}
\end{gather*}
5. So this shows: $||A^Tv-\frac{1}{s}\mathbf{1}||^2 = v_{\perp}^TAA^Tv_{\perp}$
6. Next, let $\hat{p}$ be the unit-norm (magnitude 1) principal (largest) eigenvector of $AA^T$
7. Using that, I will relate $\hat{p}$ to ${\sigma}_1$ and ${\sigma}_2$. Firstly, since $AA^T$ is irreducable (Lemma 5), then $\hat{p}$ is unique up to scaling. Then:
8. 

\begin{gather*}
    s = \mathbf{1}^T\mathbf{1} = (A^T\mathbf{1})^TA^T\mathbf{1} = \mathbf{1}^TAA^T\mathbf{1}\\
    = \mathbf{1}^T(Q \Lambda Q^T)\mathbf{1} = \mathbf{1}^T({\sigma}_1^2\hat{p}\hat{p}^T + {\sum}_{j=2}^s{\sigma}_j^2\hat{v}_j\hat{v}_j^T)\mathbf{1} = {\sigma}_1^2\mathbf{1}^T\hat{p}\hat{p}^T\mathbf{1} + {\sum}_{j=2}^s{\sigma}_j^2\mathbf{1}^T\hat{v}_j\hat{v}_j^T\mathbf{1} \\
    = {\sigma}_1^2(\hat{p}^T\mathbf{1})^T\hat{p}^T\mathbf{1} + {\sum}_{j=2}^s{\sigma}_j^2(\hat{v}_j^T\mathbf{1})^T\hat{v}_j^T\mathbf{1} =  {\sigma}_1^2(\hat{p}^T\mathbf{1})^2 + {\sum}_{j=2}^s{\sigma}_j^2(\hat{v}_j^T\mathbf{1})^2 \\
    =  {\sigma}_1^2(\mathbf{1}^T\hat{p})^2 + {\sum}_{j=2}^s{\sigma}_j^2(\mathbf{1}^T\hat{v}_j)^2 \leq {\sigma}_1^2(\mathbf{1}^T\hat{p})^2 + {\sigma}_2^2{\sum}_{j=2}^s(\mathbf{1}^T\hat{v}_j)^2
\end{gather*}
9. Since singular vectors are orthonormal, $\hat{p}$ and $\hat{v}_j \forall j>2$ form an orthonormal eigenvector basis for ${\R}^s$, so $\mathbf{1}$ has a representation $[\mathbf{1}^T\hat{p}, \mathbf{1}^T\hat{v}_2, \ldots,\mathbf{1}^T\hat{v}_s ]^T$ in that basis. So we have: $(\mathbf{1}^T\hat{p})^2 + {\sum}_{j=2}^s(\mathbf{1}^T\hat{v}_j)^2 = ||\mathbf{1}||^2 = s$
10. From that and 8. it follows that: $s \leq {\sigma}_1^2(\mathbf{1}^T\hat{p})^2 + {\sigma}_2^2{\sum}_{j=2}^s(\mathbf{1}^T\hat{v}_j)^2 = {\sigma}_1^2(\mathbf{1}^T\hat{p})^2 + {\sigma}_2^2(s - (\mathbf{1}^T\hat{p})^2)$
11. So this shows that: $\frac{1}{s}(\mathbf{1}^T\hat{p})^2 \geq \frac{1 -{\sigma}_2^2 }{{\sigma}_1^2-{\sigma}_2^2}$
12. Final step is to upperbound $v_{\perp}^TAA^Tv_{\perp}$
13. Let $\hat{p}_{\perp} := \hat{p} - \frac{1}{s}(\hat{p}^T\mathbf{1})\mathbf{1}$ be the component of $\hat{p}$ orthogonal to $\mathbf{1}$ (this is just $v_{orth} = v - proj_bv$) and $v_{\perp\!\!\!\!\!\perp} := v_{\perp} - (v_{\perp}^T\hat{p})\hat{p}$ be the component of $v_{\perp}$ orthogonal to $\hat{p}$
14. Then: <font color ='red'>Question</font> Why do these 2 terms disappear?, and why can you just move stuff around?

\begin{gather*}
    v_{\perp}^TAA^Tv_{\perp} = (v_{\perp\!\!\!\!\!\perp} + (v_{\perp}^T\hat{p})\hat{p})^TAA^T (v_{\perp\!\!\!\!\!\perp} + (v_{\perp}^T\hat{p})\hat{p}) = (v_{\perp\!\!\!\!\!\perp}^T + ((v_{\perp}^T\hat{p})\hat{p})^T)AA^T (v_{\perp\!\!\!\!\!\perp} + (v_{\perp}^T\hat{p})\hat{p})\\
    = v_{\perp\!\!\!\!\!\perp}^TAA^Tv_{\perp\!\!\!\!\!\perp} + v_{\perp\!\!\!\!\!\perp}^TAA^T(v_{\perp}^T\hat{p})\hat{p} + ((v_{\perp}^T\hat{p})\hat{p})^TAA^Tv_{\perp\!\!\!\!\!\perp} +  ((v_{\perp}^T\hat{p})\hat{p})^TAA^T(v_{\perp}^T\hat{p})\hat{p} \\
    = v_{\perp\!\!\!\!\!\perp}^TAA^Tv_{\perp\!\!\!\!\!\perp}+  \hat{p}^T(\hat{p}^Tv_{\perp})AA^T(v_{\perp}^T\hat{p})\hat{p} =  v_{\perp\!\!\!\!\!\perp}^TAA^Tv_{\perp\!\!\!\!\!\perp}+  (v_{\perp}^T\hat{p})^2\hat{p}^TAA^T\hat{p} \\
    \overset{(b)}{\leq} {\sigma}_2^2||v_{\perp\!\!\!\!\!\perp}||^2 + (v_{\perp}^T\hat{p})^2 {\sigma}_1^2\\
    \overset{(c)}{=} {\sigma}_2^2(||v_{\perp}||^2-(v_{\perp}^T\hat{p})^2) + (v_{\perp}^T\hat{p})^2 {\sigma}_1^2\\
    \overset{(d)}{=} ({\sigma}_1^2 -{\sigma}_2^2) (v_{\perp}^T\hat{p}_{\perp})^2 + {\sigma}_2^2||v_{\perp}||^2\\
    \overset{(e)}{\leq} (({\sigma}_1^2 -{\sigma}_2^2) ||\hat{p}_{\perp}||^2 + {\sigma}_2^2)||v_{\perp}||^2\\
    \overset{(f)}{=} (({\sigma}_1^2 -{\sigma}_2^2) (1-\frac{1}{s}(\mathbf{1}^T\hat{p})^2) + {\sigma}_2^2)||v_{\perp}||^2\\
    \overset{(g)}{\leq}({\sigma}_1^2 +{\sigma}_2^2 - 1)||v_{\perp}||^2
    
\end{gather*}
- where:
    - (a) was skipped
    - (b) follows from Courant-Fisher theorem and because $v_{\perp\!\!\!\!\!\perp}$ is orthogonal to principal eigenspace of $AA^T$
    - (c) follows from Pythagoras theorem
    - (d) holds because $v_{\perp}^T\mathbf{1} = 0 \Rightarrow v_{\perp}^T\hat{p} = v_{\perp}^T(\hat{p}_{\perp} + \frac{1}{s}(\hat{p}^T\mathbf{1})\mathbf{1}) = v_{\perp}^T\hat{p}_{\perp} + v_{\perp}^T\frac{1}{s}(\hat{p}^T\mathbf{1})\mathbf{1}$ ?
    - (e) holds from Cauchy-Schwartz inequality
    - (f) follows from Pythagoras theorem, and the fact that $\hat{p}_{\perp}$ is orthogonal to $\mathbf{1}$ and $||\hat{p}||=1$
    - (g) follows from point 11
    
QED

Lemma 7: Let $\{Z_i\}_{i=1}^4$ be random vectors of the same dimensions such that $\mathbb{E}[Z_1^TZ_3]=[Z_1^TZ_4]=0$.
Then:
\begin{gather*}
    \mathbb{E}||Z_1 + Z_2 + Z_3 + Z_4||^2 \leq 2\mathbb{E}||Z_2||^2 + 4\mathbb{E}||Z_2||^2 + 3(\mathbb{E}||Z_3||^2 + \mathbb{E}||Z_4||^2)
\end{gather*}

<font color='red'>Pf</font>
\begin{gather*}
    \mathbb{E}||Z_1 + Z_2 + Z_3 + Z_4||^2 \leq 2\mathbb{E}||Z_2||^2 + 4\mathbb{E}||Z_2||^2 + 3(\mathbb{E}||Z_3||^2 + \mathbb{E}||Z_4||^2) \\
    = \mathbb{E}[(Z_1 + Z_2 + Z_3 + Z_4)^T(Z_1 + Z_2 + Z_3 + Z_4)] \\
    = \mathbb{E}[(\underset{i=1}{\overset{4}{\sum}}Z_i^TZ_i)] + 2\mathbb{E}[Z_1^TZ_2] + \mathbb{E}[Z_2^TZ_3 + Z_2^TZ_4 + Z_3^TZ_4] \\ 
    = \underset{i=1}{\overset{4}{\sum}}\mathbb{E}||Z_i||^2 + 2\mathbb{E}[Z_1^TZ_2] + \mathbb{E}||Z_2 + Z_3 + Z_4||^2 - \underset{j=2}{\overset{4}{\sum}}\mathbb{E}||Z_j||^2 \\
    \overset{(a)}{\leq} \underset{i=1}{\overset{4}{\sum}}\mathbb{E}||Z_i||^2 + \mathbb{E}||Z_1||^2 + \mathbb{E}||Z_2||^2 + \mathbb{E}||Z_2 + Z_3 + Z_4||^2 - \underset{j=2}{\overset{4}{\sum}}\mathbb{E}||Z_j||^2 \\
    \overset{(b)}{\leq} \underset{i=1}{\overset{4}{\sum}}\mathbb{E}||Z_i||^2 + \mathbb{E}||Z_1||^2 + \mathbb{E}||Z_2||^2 +  3\underset{j=2}{\overset{4}{\sum}}\mathbb{E}||Z_j||^2 - \underset{j=2}{\overset{4}{\sum}}\mathbb{E}||Z_j||^2 \\
    = \underset{i=1}{\overset{4}{\sum}}\mathbb{E}||Z_i||^2 + \mathbb{E}||Z_1||^2 + \mathbb{E}||Z_2||^2 +  2\underset{j=2}{\overset{4}{\sum}}\mathbb{E}||Z_j||^2\\
    = 2\mathbb{E}||Z_1||^2 + 4 \mathbb{E}||Z_2||^2 + 3\mathbb{E}||Z_3||^2 + 3\mathbb{E}||Z_4||^2 \\ QED
\end{gather*}

Where:
- $(a)$ follows from the fact that $\mathbb{E}||Z_1 - Z_2||^2 = \mathbb{E}||Z_1||^2 - 2\mathbb{E}(Z_1^TZ_2) +  \mathbb{E}||Z_2||^2 \geq 0 \iff \mathbb{E}||Z_1||^2 + \mathbb{E}||Z_2||^2 \geq 2\mathbb{E}(Z_1^TZ_2)$
- $(b)$ follows from lemma 3


Lemma 8: Given $k \in [T]$, $\{h_q:{\R}^p \leftarrow {\R}^p\}_{q=0}^{k-1}$ = a clollection of non-random vector valued functions, then: 
\begin{gather*}
    \mathbb{E}[(\overset{\sim}{\nabla}f_i(x_i^{(t,q)})-\nabla f_i(x_i^{(t,q)}))^T\overset{\sim}{\nabla}f_i(x_i^{(t,r)})-\nabla f_i(x_i^{(t,r)})] = 0 \\ 
    \forall i \in [n]  \And \forall q,r \in {0, 1, \ldots, k - 1}, q \neq r 
\end{gather*}

<font color = 'red'>Pf</font>
\begin{gather*}
    \mathbb{E}[(\overset{\sim}{\nabla}f_i(x_i^{(t,q)})-\nabla f_i(x_i^{(t,q)}))^T(\overset{\sim}{\nabla}f_i(x_i^{(t,r)})-\nabla f_i(x_i^{(t,r)}))] \\
    \overset{(a)}{=} \mathbb{E}[\mathbb{E}[(\overset{\sim}{\nabla}f_i(x_i^{(t,q)})-\nabla f_i(x_i^{(t,q)}))^T(\overset{\sim}{\nabla}f_i(x_i^{(t,r)})-\nabla f_i(x_i^{(t,r)}))|x_i^{(t, q)}, x_i^{(t, r)}, \overset{\sim}{\nabla}f_i(x_i^{(t,r)})]] \\
    \overset{(b)}{=} \mathbb{E}[\mathbb{E}[(\overset{\sim}{\nabla}f_i(x_i^{(t,q)})-\nabla f_i(x_i^{(t,q)}))^T|x_i^{(t, q)}, x_i^{(t, r)}, \overset{\sim}{\nabla}f_i(x_i^{(t,r)})](\overset{\sim}{\nabla}f_i(x_i^{(t,r)})-\nabla f_i(x_i^{(t,r)}))] \\ 
    = \mathbb{E}[\mathbb{E}[(\overset{\sim}{\nabla}f_i(x_i^{(t,q)})-\nabla f_i(x_i^{(t,q)}))|x_i^{(t, q)}, x_i^{(t, r)}, \overset{\sim}{\nabla}f_i(x_i^{(t,r)})]^T(\overset{\sim}{\nabla}f_i(x_i^{(t,r)})-\nabla f_i(x_i^{(t,r)}))] \\ 
    \overset{(c)}{=} \mathbb{E}[\mathbf{0}(\overset{\sim}{\nabla}f_i(x_i^{(t,r)})-\nabla f_i(x_i^{(t,r)}))] = 0 \\ 
    QED
\end{gather*}
where:
- $(a)$ follows from the iterated expectation theorem
- $(b)$ follows from the fact that conditioning on the specified values makes the right hand side constant, so I can take it out of the expectation 
- $(c)$ follows from Assumption 3 aboud unbiasedness of the stochastic gradient

Lemma 9: Suppose ${\eta}_{t} \leq \frac{1}{\sqrt{2}T\beta} \forall t \geq 0$ (learning rate constraint). Also let ${\beta}^{(t,0)} = x^{(t)}$ and ${\beta}^{(t,k)} = {\beta}^{(t,k-1)} - {\eta}_{t}\frac{1}{n}\underset{i = 0}{\overset{n}{\sum}} \nabla f_i({\beta}^{(t,k-1)})$ be the theoretical model that would result from using all clients and their true gradients. Then $\forall i \in [n]$ we have:
\begin{gather*}
    \underset{k=0}{\overset{r-1}{\sum}}\mathbb{E}||x_i^{(t, k)- {\beta}^{(t, k)}}||^2 \leq 2eT^2 {\eta}_{t}^2 ({\varrho}^2 + 2{\delta}^2 + 8 {\beta}^2 \mathbb{E}||x^{(t)}-x^*||^2) 
\end{gather*}

<font color = 'red'>Pf</font>
Let $a^{(t, k)} := \mathbb{E}||x_i^{(t, k)} - {\beta}^{(t, k)} ||^2$